In [5]:
# IMporting Dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [6]:
# Creating a sqlite databse file in the exisitng folder
engine = create_engine('sqlite://', echo=False)
conn = sqlite3.connect("covid.sqlite")

In [22]:
# Using CDC provided api's to load data in dataframe 
# The link is to the CDC's covid tracker file that is updated weekly
json_file = "https://data.cdc.gov/resource/n8mc-b4w4.json"
covid_df = pd.read_json(json_file)
# Dropping columns that are not required
covid_df=covid_df.drop(columns=['county_fips_code', 'state_fips_code',"sex","race","ethnicity","case_positive_specimen","case_onset_interval","process","exposure_yn"])
#Pushing data to the sqlite databse file
covid_df.to_sql('covid_tracker', con=conn, if_exists="replace")

In [21]:
#The following json file shows vaccine adminstration data for each State and also provides population statistics for the same
json_file = "https://data.cdc.gov/resource/8xkx-amqh.json"
admin = pd.read_json(json_file)
admin=admin.drop(columns=["fips", "mmwr_week", "recip_county", "completeness_pct","series_complete_pop_pct_svi","series_complete_12pluspop_pct_svi","series_complete_18pluspop_pct_svi","series_complete_65pluspop_pct_svi"])
admin=admin.rename(columns={"recip_state":"State", 
                            "series_complete_pop_pct":"%_Population_Vaccinated", 
                            "series_complete_yes":"Total_Population_Vaccinated",
                            "series_complete_12plus":"Vaccinated_Population_Age12to17", 
                            "series_complete_12pluspop":"%_Population_Vaccinated_Age12to17", 
                            "series_complete_18plus":"Vaccinated_Population_Age18to64",
                            "series_complete_18pluspop":"%_Population_Vaccinated_Age18to64",
                            "series_complete_65plus":"Vaccinated_Population_Age65andup",
                            "series_complete_65pluspop":"%_Population_Vaccinated_Age65andup",
                            "administered_dose1_recip":"1st_dose_administered",
                            "administered_dose1_pop_pct":"%_Population_1st_dose_administered",
                            "administered_dose1_recip_12plus":"Population_1st_dose_administered_Age12to17",
                            "administered_dose1_recip_12pluspop_pct":"%_Population_1st_dose_administered_Age12to17",
                            "administered_dose1_recip_18plus":"Population_1st_dose_administered_Age18to64",
                            "administered_dose1_recip_18pluspop_pct":"%_Population_1st_dose_administered_Age18to64",
                            "administered_dose1_recip_65plus":"Population_1st_dose_administered_Age65andup",
                            "administered_dose1_recip_65pluspop_pct":"%_Population_1st_dose_administered_Age65andup",
                            "svi_ctgy":"Social_Vulnerability_Index"})
admin=admin.groupby('State').sum()
admin=admin.reset_index()
admin.to_sql('vaccine_administration', con=conn, if_exists="replace")

In [9]:
# The following is the moderna vaccine allocation by state file provided by the CDC
json_file = "https://data.cdc.gov/resource/b7pe-5nws.json"
moderna_df = pd.read_json(json_file)
moderna_df=moderna_df.rename(columns={"_1st_dose_allocations":"Moderna_1st_allocations", "_2nd_dose_allocations":"Moderna_2nd_allocations"})
# Grouping by jurisdiction to obtain the total allocation of vaccine
moderna_df=moderna_df.groupby(["jurisdiction"]).sum()

In [10]:
# The following is the pfizer vaccine allocation by state file provided by the CDC
json_file = "https://data.cdc.gov/resource/saz5-9hgg.json"
pfizer_df = pd.read_json(json_file)
pfizer_df=pfizer_df.rename(columns={"_1st_dose_allocations":"Pfizer_1st_allocations", "_2nd_dose_allocations":"Pfizer_2nd_allocations"})
pfizer_df=pfizer_df.groupby(["jurisdiction"]).sum()

In [11]:
# The following is the Johnson vaccine allocation by state file provided by the CDC
json_file = "https://data.cdc.gov/resource/w9zu-fywh.json"
jnj = pd.read_json(json_file)
jnj=jnj.rename(columns={"_1st_dose_allocations":"J&J_1st_allocations"})
jnj=jnj.groupby(["jurisdiction"]).sum()

In [25]:
#Merging moderna and pfizer and Johnson dataframes to show total allocations
vaccine_df=pfizer_df.merge(moderna_df, how="outer", on="jurisdiction")
vaccine_df=vaccine_df.merge(jnj,how="outer", on="jurisdiction")
vaccine_df=vaccine_df.reset_index()
vaccine_df=vaccine_df.rename(columns={"jurisdiction":"state"})
vaccine_df.to_sql('vaccine_allocation', con=conn, if_exists="replace")

In [26]:
# The following table show the number of deaths caused by covid and biforcates data by age group and underlying conditions and also provides the name of the state in which the tragedy occured

json_file = "https://data.cdc.gov/resource/hk9y-quqm.json"
cond_1_df = pd.read_json(json_file)
cond_2_df = cond_1_df.drop(columns=["data_as_of", "start_date", "end_date", "group", "icd10_codes", "flag", "number_of_mentions"])
cond_2_df.to_sql('covid_death_tracker', con=conn, if_exists="replace")

In [29]:
# The following csv is for the population records of America obtained from the census bureau website
csv_file = "population.csv"
population_df = pd.read_csv(csv_file)
pop_1_df=population_df.drop(columns=["growthSince2010", "Pop2010", "Pop2018"])
pop_2_df=pop_1_df.rename(columns={"Percent":"percent_of_united_states", "density":"density_per_square_mile"})
pop_2_df.to_sql('population', con=conn, if_exists="replace")

In [28]:
# the following is a json file that includes the state names and their abbreviateions
json_file = "data.json"
states_df = pd.read_json(json_file)
states_clean_df = states_df.drop(columns="Abbrev")
states_clean_df.to_sql('state_names', con=conn, if_exists="replace")

In [22]:
# verifying data pushed to sqlite file
engine = create_engine("sqlite:///covid.sqlite")
engine.execute("Select * From state_names").fetchall()

[(0, 'Alabama', 'AL'),
 (1, 'Alaska', 'AK'),
 (2, 'Arizona', 'AZ'),
 (3, 'Arkansas', 'AR'),
 (4, 'California', 'CA'),
 (5, 'Colorado', 'CO'),
 (6, 'Connecticut', 'CT'),
 (7, 'Delaware', 'DE'),
 (8, 'District of Columbia', 'DC'),
 (9, 'Florida', 'FL'),
 (10, 'Georgia', 'GA'),
 (11, 'Hawaii', 'HI'),
 (12, 'Idaho', 'ID'),
 (13, 'Illinois', 'IL'),
 (14, 'Indiana', 'IN'),
 (15, 'Iowa', 'IA'),
 (16, 'Kansas', 'KS'),
 (17, 'Kentucky', 'KY'),
 (18, 'Louisiana', 'LA'),
 (19, 'Maine', 'ME'),
 (20, 'Maryland', 'MD'),
 (21, 'Massachusetts', 'MA'),
 (22, 'Michigan', 'MI'),
 (23, 'Minnesota', 'MN'),
 (24, 'Mississippi', 'MS'),
 (25, 'Missouri', 'MO'),
 (26, 'Montana', 'MT'),
 (27, 'Nebraska', 'NE'),
 (28, 'Nevada', 'NV'),
 (29, 'New Hampshire', 'NH'),
 (30, 'New Jersey', 'NJ'),
 (31, 'New Mexico', 'NM'),
 (32, 'New York', 'NY'),
 (33, 'North Carolina', 'NC'),
 (34, 'North Dakota', 'ND'),
 (35, 'Ohio', 'OH'),
 (36, 'Oklahoma', 'OK'),
 (37, 'Oregon', 'OR'),
 (38, 'Pennsylvania', 'PA'),
 (39, 'Rhode Is